# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 40 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.01340


extracting tarball to tmp_2302.01340...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.01345


extracting tarball to tmp_2302.01345...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.01836


extracting tarball to tmp_2302.01836...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.01848


extracting tarball to tmp_2302.01848...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01678-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01678) | **A complex dust morphology in the high-luminosity AGN Mrk 876**  |
|| <mark>Hermine Landt</mark>, et al. -- incl., <mark>Jörg-Uwe Pott</mark>, <mark>Juan V. Hernández Santisteban</mark> |
|*Appeared on*| *2023-02-06*|
|*Comments*| *18 pages, 7 figures; accepted to ApJ*|
|**Abstract**| Recent models for the inner structure of active galactic nuclei (AGN) advocate the presence of a radiatively accelerated, dusty outflow launched from the outer regions of the accretion disk. Here we present the first near-infrared (near-IR) variable (rms) spectrum for the high-luminosity, nearby AGN Mrk 876. We find that it tracks the accretion disk spectrum out to longer wavelengths than the mean spectrum due to a reduced dust emission. The implied outer accretion disk radius is consistent with the infrared results predicted by a contemporaneous optical accretion disk reverberation mapping campaign and much larger than the self-gravity radius. The reduced flux variability of the hot dust could be either due to the presence of a secondary, constant dust component in the mean spectrum or introduced by the destructive superposition of the dust and accretion disk variability signals or some combination of both. Assuming thermal equilibrium for optically thin dust, we derive the luminosity-based dust radius for different grain properties using our measurement of the temperature. We find that in all cases considered the values are significantly larger than the dust response time measured by IR photometric monitoring campaigns, with the least discrepancy present relative to the result for a wavelength-independent dust emissivity law, i.e. a blackbody, which is appropriate for large grain sizes. This result can be well explained by assuming a flared, disk-like structure for the hot dust. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01818-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01818) | **Baryon-photon interactions in Resummed Kinetic Field Theory**  |
|| Ivan Kostyuk, <mark>Robert Lilow</mark>, Matthias Bartelmann |
|*Appeared on*| *2023-02-06*|
|*Comments*| *16 pages, 3 figures*|
|**Abstract**| We explore how interactions between baryons and photons can be incorporated into Kinetic Field Theory (KFT), a description of cosmic structure formation based on classical Hamiltonian particle dynamics. In KFT, baryons are described as effective mesoscopic particles which represent fluid elements governed by the hydrodynamic equations. In this paper, we modify the mesoscopic particle model to include pressure effects exerted on baryonic matter through interactions with photons. As a proof of concept, we use this extended mesoscopic model to describe the tightly coupled baryon-photon fluid between matter-radiation equality and recombination. We show that this model can qualitatively reproduce the formation of baryon-acoustic oscillations in the cosmological power spectrum. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01340-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01340) | **The PDR fronts in M17-SW localized with FIFI-LS onboard SOFIA**  |
|| <mark>Randolf Klein</mark>, et al. -- incl., <mark>Christian Fischer</mark> |
|*Appeared on*| *2023-02-06*|
|*Comments*| *18 pages, 17 figures, accepted by ApJ*|
|**Abstract**| To understand star formation rates, studying feedback mechanisms that regulate star formation is necessary. The radiation emitted by nascent massive stars play a significant role in feedback by photo-dissociating and ionizing their parental molecular clouds. To gain a detailed picture of the physical processes, we mapped the photo-dissociation region (PDR) M17-SW in several fine structure and high-J CO lines with FIFI-LS, the far-infrared imaging spectrometer aboard SOFIA. An analysis of the CO and [O I]146$\mu$m line intensities, combined with the far infrared intensity, allows us to create a density and UV intensity map using a one dimensional model. The density map reveals a sudden change in the gas density crossing the PDR. The strengths and limits of the model and the locations of the ionization and photo-dissociation front of the edge-on PDR are discussed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01804-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01804) | **SRG/eROSITA-triggered XMM-Newton observations of three Be/X-ray binaries  in the LMC: Discovery of X-ray pulsations**  |
|| F. Haberl, et al. -- incl., <mark>P. Maggi</mark> |
|*Appeared on*| *2023-02-06*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| Using data from eROSITA, the soft X-ray instrument aboard Spectrum-Roentgen-Gamma (SRG), we report the discovery of two new hard transients, eRASSU J050810.4-660653 and eRASSt J044811.1-691318, in the Large Magellanic Cloud. We also report the detection of the Be/X-ray binary RX J0501.6-7034 in a bright state. We initiated follow-up observations to investigate the nature of the new transients and to search for X-ray pulsations coming from RX J0501.6-7034. We analysed the X-ray spectra and light curves from our XMM-Newton observations, obtained optical spectra using the South African Large Telescope to look for Balmer emission lines and utilised the archival data from the Optical Gravitational Lensing Experiment (OGLE) for the long-term monitoring of the optical counterparts. We find X-ray pulsations for eRASSU J050810.4-660653, RX J0501.6-7034, and eRASSt J044811.1-691318 of 40.6 s, 17.3 s, and 784 s, respectively. The Halpha emission lines with equivalent widths of -10.4 A (eRASSU J050810.4-660653) and -43.9 A (eRASSt J044811.1-691318) were measured, characteristic for a circumstellar disc around Be stars. The OGLE I- and V-band light curves of all three systems exhibit strong variability. A regular pattern of deep dips in the light curves of RX J0501.6-7034 suggests an orbital period of ~451 days. We identify the two new hard eROSITA transients eRASSU J050810.4-660653 and eRASSt J044811.1-691318 and the known Be/X-ray binary RX J0501.6-7034 as Be/X-ray binary pulsars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01836-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01836) | **Fermi-LAT Gamma-ray Emission Discovered from the Composite Supernova  Remnant B0453-685 in the Large Magellanic Cloud**  |
|| Jordan Eagle, et al. -- incl., <mark>Peter Mahhov</mark>, <mark>Joseph Gelfand</mark> |
|*Appeared on*| *2023-02-06*|
|*Comments*| *18 pages, 8 figures, accepted for publication in ApJ Feb. 1, 2023*|
|**Abstract**| We report the second extragalactic pulsar wind nebula (PWN) to be detected in the MeV-GeV band by the Fermi-LAT, located within the Large Magellanic Cloud (LMC). The only other known PWN to emit in the Fermi band outside of the Milky Way Galaxy is N 157B which lies to the west of the newly detected gamma-ray emission at an angular distance of 4 degrees. Faint, point-like gamma-ray emission is discovered at the location of the composite supernova remnant (SNR) B0453-685 with a ~ 4 sigma significance from energies 300 MeV - 2 TeV. We present the Fermi-LAT data analysis of the new gamma-ray source, coupled with a detailed multi-wavelength investigation to understand the nature of the observed emission. Combining the observed characteristics of the SNR and the physical implications from broadband modeling, we argue it is unlikely the SNR is responsible for the gamma-ray emission. While the gamma-ray emission is too faint for a pulsation search, we try to distinguish between any pulsar and PWN component of SNR B0453-685 that would be responsible for the observed gamma-ray emission using semi-analytic models. We determine the most likely scenario is that the old PWN (t ~ 14,000 years) within B0453-685 has been impacted by the return of the SNR reverse shock with a possible substantial pulsar component below 5 GeV. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01848-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01848) | **Dashing through the cluster: An X-ray to radio view of UGC 10420  undergoing ram-pressure stripping**  |
|| <mark>Smriti Mahajan</mark>, Kulinder Pal Singh, Juhi Tiwari, Somak Raychaudhury |
|*Appeared on*| *2023-02-06*|
|*Comments*| *Accepted for publication in the Publication of the Astronomical Society of Australia (PASA). 16 pages, 11 figures*|
|**Abstract**| We present multi-wavelength data and analysis, including new FUV AstroSat/UVIT observations of the spiral galaxy UGC 10420 (z=0.032), a member of the cluster Abell 2199. UGC 10420 is present on the edge of the X-ray emitting region of the cluster at a distance of ~ 680 kpc from the centre. The FUV data shows intense knots of star formation on the leading edge of the galaxy, accompanied by a tail of the same on the diametrically opposite side. Our analysis shows that the images of the galaxy disk in the optical and mid-infrared are much smaller in size than that in the FUV. While the broadband optical colours of UGC 10420 are typical of a post-starburst galaxy, the SFR derived from a UV-to-IR spectral energy distribution is at least a factor of nine higher than that expected for a star-forming field galaxy of similar mass at its redshift. A careful removal of the contribution of the diffuse intracluster gas shows that the significant diffuse X-ray emission associated with the inter-stellar medium of UGC 10420 has a temperature, T_X = 0.24^{+0.09}_{-0.06} keV (0.4-2.0 keV) and luminosity, L_X = 1.8+/-0.9 x 10^{40} erg/s, which are typical of the X-ray emission from late-type spiral galaxies. Our analysis favours a scenario where the interaction of a galaxy with the hot intra-cluster medium of the cluster, perturbs the gas in the galaxy causing starburst in the leading edge of the disk. On the other hand, the turbulence thus developed may also push some of the gas out of the disk. Interactions between the gas ejected from the galaxy and the intracluster medium can then locally trigger star formation in the wake of the galaxy experiencing ram-pressure stripping. Our data however does not rule out the possibility of a flyby encounter with a neighbouring galaxy, although no relevant candidates are observed in the vicinity of UGC 10420. (abridged) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01420-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01420) | **Multicolour Optical Variability Monitoring of Blazars with High Time  Resolution**  |
|| X. Chang, et al. -- incl., <mark>C. X. Liu</mark>, <mark>H. Z. Li</mark>, <mark>Y. Li</mark> |
|*Appeared on*| *2023-02-06*|
|*Comments*| *18 pages, 8 figures, accepted in MNRAS*|
|**Abstract**| We carried out a high time-resolution, multicolour optical observing campaign for eight $\gamma$-ray detected blazars during 2010-2020. We analyze flux variations, correlations between magnitudes and colours on different timescales. Intraday variability (IDV) is detected in all eight sources of our sample. A bluer-when-brighter (BWB) chromatic trend is dominant on intraday timescales. On the short timescales, the BWB trend only shows up in ON 231, 3C 279, BL Lacertae and 1E 1458.8+2249. There is a BWB trend in 3C 279 on the long timescale. We estimate the upper limits of black hole mass for three blazars (i.e. ON 321, 1ES 1426+42.8, PKS 1510-089) using variability timescales. On April 13, 2010 a potential quasi-periodic oscillation (QPO) with the period of $P=48.67\pm13.90$ minutes is found in 1ES 1426+42.8. The light curve on March 16, 2021 further shows the existence of the QPO phenomenon. The QPO in this target deserves further observation and confirmation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01794-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01794) | **VPNEP: Detailed characterization of TESS targets around the Northern  Ecliptic Pole**  |
|| K. G. Strassmeier, et al. -- incl., <mark>M. Mallonn</mark>, <mark>D. Brown</mark> |
|*Appeared on*| *2023-02-06*|
|*Comments*| **|
|**Abstract**| We embarked on a high-resolution optical spectroscopic survey of bright Transiting Exoplanet Survey Satellite (TESS) stars around the Northern Ecliptic Pole (NEP), dubbed the Vatican-Potsdam-NEP (VPNEP) survey. Our NEP coverage comprises 1067 stars, of which 352 are bona fide dwarf stars and 715 are giant stars, all cooler than spectral type F0 and brighter than V=8. m 5. Our aim is to characterize these stars for the benefit of future studies in the community. We analyzed the spectra via comparisons with synthetic spectra. Particular line profiles were analyzed by means of eigen-profiles, equivalent widths, and relative emission-line fluxes (when applicable). Two R=200 000 spectra were obtained for each of the dwarf stars with the Vatican Advanced Technology Telescope (VATT) and the Potsdam Echelle Polarimetric and Spectroscopic Instrument (PEPSI), with typically three R=55 000 spectra obtained for the giant stars with STELLA and the STELLA Echelle Spectrograph (SES). Combined with V-band magnitudes, Gaia eDR3 parallaxes, and isochrones from the Padova and Trieste Stellar Evolutionary Code, the spectra can be used to obtain radial velocities, effective temperatures, gravities, rotational and turbulence broadenings, stellar masses and ages, and abundances for 27 chemical elements, as well as isotope ratios for lithium and carbon, line bisector spans, convective blue-shifts (when feasible), and levels of magnetic activity from H{\alpha}, H{\beta}, and the Ca ii infrared triplet. In this initial paper, we discuss our analysis tools and biases, presenting our first results from a pilot sub-sample of 54 stars (27 bona-fide dwarf stars observed with VATT+PEPSI and 27 bona-fide giant stars observed with STELLA+SES) and making all reduced spectra available to the community. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xed in position 122643: invalid continuation byte</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.01678.md
    + _build/html/tmp_2302.01678/./mrk876irsed.png
    + _build/html/tmp_2302.01678/./rmsmeanmrk876irsedpanels2.png
    + _build/html/tmp_2302.01678/./rmsmeanmrk876irsedpanels3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\la}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{<}}}}$
$\newcommand{\ga}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{>}}}}$
$\newcommand{\OIII}{[O~{\sevenrm III}]}$
$\newcommand{\FeII}{Fe~{\sevenrm II}}$
$\newcommand{\FeIIf}{[Fe~{\sevenrm II}]}$
$\newcommand{\SIII}{[S~{\sevenrm III}]}$
$\newcommand{\HeI}{He~{\sevenrm I}}$
$\newcommand{\HeII}{He~{\sevenrm II}}$
$\newcommand{\NeV}{[Ne~{\sevenrm V}]}$
$\newcommand{\OIV}{[O~{\sevenrm IV}]}$
$\newcommand{\iraf}{{\sevenrm IRAF}}$
$\newcommand{\mpfit}{{\sevenrm MPFIT}}$
$\newcommand{\galfit}{{\sevenrm GALFIT}}$
$\newcommand{\prepspec}{{\sevenrm PrepSpec}}$
$\newcommand{\mapspec}{{\sevenrm mapspec}}$
$\newcommand{\cream}{{\sevenrm CREAM}}$
$\newcommand{\javelin}{{\sevenrm JAVELIN}}$
$\newcommand{\cloudy}{{\sevenrm CLOUDY}}$
$\newcommand{\banzai}{{\sevenrm BANZAI}}$
$\newcommand{\orac}{{\sevenrm ORAC}}$
$\newcommand{\demc}{{\sevenrm DEMC}}$
$\newcommand{\gp}{\mathcal{GP}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\la$}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{<}}}}$
$\newcommand{$\ga$}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{>}}}}$
$\newcommand{$\OIII$}{[O~{\sevenrm III}]}$
$\newcommand{$\FeII$}{Fe~{\sevenrm II}}$
$\newcommand{$\FeII$f}{[Fe~{\sevenrm II}]}$
$\newcommand{$\SIII$}{[S~{\sevenrm III}]}$
$\newcommand{$\HeI$}{He~{\sevenrm I}}$
$\newcommand{$\HeI$I}{He~{\sevenrm II}}$
$\newcommand{$\NeV$}{[Ne~{\sevenrm V}]}$
$\newcommand{$\OIV$}{[O~{\sevenrm IV}]}$
$\newcommand{$\iraf$}{{\sevenrm IRAF}}$
$\newcommand{$\mpfit$}{{\sevenrm MPFIT}}$
$\newcommand{$\ga$lfit}{{\sevenrm GALFIT}}$
$\newcommand{$\prepspec$}{{\sevenrm PrepSpec}}$
$\newcommand{$\mapspec$}{{\sevenrm mapspec}}$
$\newcommand{$\cream$}{{\sevenrm CREAM}}$
$\newcommand{$\javelin$}{{\sevenrm JAVELIN}}$
$\newcommand{$\cloudy$}{{\sevenrm CLOUDY}}$
$\newcommand{$\banzai$}{{\sevenrm BANZAI}}$
$\newcommand{$\orac$}{{\sevenrm ORAC}}$
$\newcommand{$\demc$}{{\sevenrm DEMC}}$
$\newcommand{$\gp$}{\mathcal{GP}}$</div>



<div id="title">

# A complex dust morphology in the high-luminosity AGN Mrk 876

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.01678-b31b1b.svg)](https://arxiv.org/abs/2302.01678)<mark>Appeared on: 2023-02-06</mark> - _18 pages, 7 figures; accepted to ApJ_

</div>
<div id="authors">

<mark>Hermine Landt</mark>, et al. -- incl., <mark>J\"org-Uwe Pott</mark>, <mark>Juan V. Hernández Santisteban</mark>

</div>
<div id="abstract">

**Abstract:** Recent models for the inner structure of active galactic nuclei (AGN) advocate the presence of a radiatively accelerated, dusty outflow launched from the outer regions of the accretion disk. Here we present the first near-infrared (near-IR) variable (rms) spectrum for the high-luminosity, nearby AGN Mrk 876. We find that it tracks the accretion disk spectrum out to longer wavelengths than the mean spectrum due to a reduced dust emission. The implied outer accretion disk radius is consistent with the infrared results predicted by a contemporaneous optical accretion disk reverberation mapping campaign and much larger than the self-gravity radius. The reduced flux variability of the hot dust could be either due to the presence of a secondary, constant dust component in the mean spectrum or introduced by the destructive superposition of the dust and accretion disk variability signals or some combination of both. Assuming thermal equilibrium for optically thin dust, we derive the luminosity-based dust radius for different grain properties using our measurement of the temperature. We find that in all cases considered the values are significantly larger than the dust response time measured by IR photometric monitoring campaigns, with the least discrepancy present relative to the result for a wavelength-independent dust emissivity law, i.e. a blackbody, which is appropriate for large grain sizes. This result can be well explained by assuming a flared, disk-like structure for the hot dust.

</div>

<div id="div_fig1">

<img src="tmp_2302.01678/./mrk876irsed.png" alt="Fig2" width="100%"/>

**Figure 2. -** $\la$bel{irsed} Gemini GNIRS near-IR spectrum from 2016 May 25 shown as luminosity versus rest-frame wavelength (black spectrum). We decomposed the continuum into two components, namely, an accretion disk spectrum that approximates the wavelength range $\la 1 \mu$m (black dotted curve) and still dominates at $1 \mu$m (vertical dashed line) and hot dust emission (red spectrum). We fitted the hot dust continuum with a blackbody spectrum (red dotted curve) and modified blackbody spectra for carbon and silicate dust (green and blue dotted curves, respectively) with resulting best-fit temperatures as listed in Table \ref{lumradiustab}. (*irsed*)

</div>
<div id="div_fig2">

<img src="tmp_2302.01678/./rmsmeanmrk876irsedpanels2.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\la$bel{fracvar} Top panel: The flux difference between the normalised mean and variable spectra (blue) can be attributed to an additional (non-variable) dust component with a blackbody temperature of $T \sim 1300$ K (blue dotted line). The template of an elliptical host galaxy is also shown (magenta). Bottom panel: The fractional amplitude of the variations as a percentage of the mean flux versus the rest-frame wavelength. (*fracvar*)

</div>
<div id="div_fig3">

<img src="tmp_2302.01678/./rmsmeanmrk876irsedpanels3.png" alt="Fig6" width="100%"/>

**Figure 6. -** $\la$bel{rmsspec} Top panel: The mean (black) and variable near-IR spectrum (red) for our campaign normalised at rest-frame $0.9 \mu$m. The spectrum of a standard accretion disk (black dotted line) is traced to $\sim 1.2 \mu$m in the variable component and only to $\sim 1 \mu$m in the mean spectrum. Bottom panel: The spectral decomposition of the variable near-IR spectrum (black) into an accretion disk spectrum (black dotted line) and a hot dust component (red spectrum) yields a blackbody temperature of $T \sim 1200$ K for the latter (red dotted curve). (*rmsspec*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

189  publications in the last 7 days.
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers